# Brief

The model "meta-llama/Llama-2-7b-chat-hf" represents a type of Large Language Model (LLM), specifically tailored towards tasks related to text generation and conversation, as hinted by the "chat-hf" suffix in its name. Although it's not specifically designed for sentiment analysis, advanced language models like this can be adapted for a variety of Natural Language Processing (NLP) tasks, including sentiment analysis.

Models like "meta-llama/Llama-2-7b" are trained on vast amounts of text, enabling them to have a sophisticated understanding of context. They can grasp nuances, ambiguities, and different language styles, which is crucial for sentiment analysis.

In this notebook, the task is to verify the presence of attributes/characteristics of audit reports (context) based on certain elements of analysis (prompt). This type of sentiment analysis is aimed at verifying the presence or non-presence of these characteristics and therefore mixes sentiment analysis with context analysis.

In [15]:
import pandas as pd

rtrs_df = pd.read_csv('/kaggle/input/rtrs-brazil-public-audit-reports-2023/brazil_rtrs.csv', encoding = 'iso-8859-14')
prompt_df = pd.read_csv('/kaggle/input/rtrs-brazil-public-audit-reports-2023/prompt_eng_rtrs.csv')

# Install necessary repositories (transformers, langchain, ...) and login in huggingface

In [2]:
!pip install -q transformers einops accelerate langchain bitsandbytes


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.3 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.12.1 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have distributed 2023.12.1 which is incompatible.
dask-cuda 23.8.0 requires dask==2023.7.1, but you have dask 2023.12.1 which is incompatible.
dask-cuda 23.8.0 requires distributed==2023.7.1, but you have distributed 2023.12.1 which is incompatible.
dask-cuda 23.8.0 requires pandas<1.6.0dev0,

 # Login Hugginface

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_hf = user_secrets.get_secret("cli-hf")
secret_wb = user_secrets.get_secret("wandb")

In [4]:
import wandb

wandb.login(key=secret_wb)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
wandb.init(project='rtrs-brasil', entity='cleytonacandeira')

wandb: Currently logged in as: cleytonacandeira. Use `wandb login --relogin` to force relogin


In [6]:
from huggingface_hub import HfApi, HfFolder

if secret_hf:
    HfFolder.save_token(secret_hf)
    api = HfApi()
    user = api.whoami(token=secret_hf)
    print(f"Logged in as: {user['name']}")
else:
    print("Token not found. Make sure it is set as an secret in add-ons.")

Logged in as: cleytoncandeira


# Pipeline

In [7]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,

    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [8]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [9]:
from langchain import PromptTemplate,  LLMChain

template = """
            Given the environmental audit criteria and observations, identify key features and concepts.
            Also, evaluate if these features are present in the provided audit evaluation text.
            Use the format ['feature', 'yes/no'] for the response, where 'yes' indicates presence and 'no' indicates absence.
            Criteria and Observations: ```{criteria_observations}```
            Audit Evaluation Text: ```{audit_text}```
           """

prompt = PromptTemplate(template=template, input_variables=["criteria_observations", "audit_text"])

llm_chain_example = LLMChain(prompt=prompt, llm=llm)


# Example

In [16]:
criteria_observations = """1.1.1 There is demonstrable knowledge of responsibilities under applicable laws;
                        1.1.2 Applicable laws are being complied with;
                        1.1.3 Producers must not engage in any act of corruption, extortion or embezzlement,
                        or in any form of bribery - including (but not limited to) promising, offering, giving or accepting any undue inducement, whether monetary or otherwise."""

audit_text = """Evidenced through documentary evaluation and interviews with managers, that the farm has a control system based on a table which contains the
                    \x93applicable legislation and applicable laws Santa Cruz Farm\x94, containing Norms, Decrees , Laws and other legislation pertinent to the operation
                  of the farm and maintains a service provision contract with third-party companies, responsible for maintaining licenses and other mandatory regulatory
                  documents in accordance with applicable legislation.
              The farm has implemented an anti-corruption policy and it is disseminated among all employees in key sectors."""

output = llm_chain_example.invoke({"criteria_observations": criteria_observations, "audit_text": audit_text})

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [17]:
print(output)

{'criteria_observations': '1.1.1 There is demonstrable knowledge of responsibilities under applicable laws;\n                        1.1.2 Applicable laws are being complied with;\n                        1.1.3 Producers must not engage in any act of corruption, extortion or embezzlement,\n                        or in any form of bribery - including (but not limited to) promising, offering, giving or accepting any undue inducement, whether monetary or otherwise.', 'audit_text': 'Evidenced through documentary evaluation and interviews with managers, that the farm has a control system based on a table which contains the\n                    \x93applicable legislation and applicable laws Santa Cruz Farm\x94, containing Norms, Decrees , Laws and other legislation pertinent to the operation\n                  of the farm and maintains a service provision contract with third-party companies, responsible for maintaining licenses and other mandatory regulatory\n                  documents in 

# Analysis RTRS Dataset

In [10]:
batch_size = 32

In [11]:
def process_batch(batch, llm_chain, prompt):
    batch_outputs = []
    for obs in batch:
        output = llm_chain.invoke({"criteria_observations": prompt, "audit_text": obs})
        batch_outputs.append(output)
    return batch_outputs

In [12]:
from tqdm import tqdm

result_llama2_df = pd.DataFrame(columns=rtrs_df.iloc[64:, 12:].columns)

for idx, col in tqdm(enumerate(rtrs_df.iloc[:, 12:].columns), total=len(rtrs_df.iloc[:, 12:].columns)):
    prompt_text = prompt_df.iloc[idx]['Describe']
    
    observations = rtrs_df[col][:64]

    current_batch = []
    column_results = []

    for obs in observations:
        formatted_input = {
            "criteria_observations": prompt_text,
            "audit_text": obs
        }
        current_batch.append(template.format(**formatted_input))

        if len(current_batch) == batch_size:
            batch_outputs = pipeline(current_batch)
            column_results.extend([output[0]['generated_text'] for output in batch_outputs])
            current_batch = []

    if current_batch:
        batch_outputs = pipeline(current_batch)
        column_results.extend([output[0]['generated_text'] for output in batch_outputs])

    result_llama2_df[col] = column_results
    wandb.log({f'{col}_processed': len(column_results)})


 18%|█▊        | 5/28 [1:28:43<7:07:13, 1114.50s/it]/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
 25%|██▌       | 7/28 [2:07:53<6:41:30, 1147.16s/it]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1363: UserWarning: Input length of input_ids is 1020, but `max_length` is set to 1000. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1363: UserWarning: Input length of input_ids is 1353, but `max_length` is set to 1000. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1363: UserWarning: Input length of input_ids is 1026, but `max_length` is set t

In [14]:
result_llama2_df.to_csv('result_llama2_df.csv')